# Conversão de Redes Neurais totalmente conectadas para Árvores de decisão

Bibliotecas sendo usadas

In [1]:
import tensorflow as tf
import numpy as np
from ArvoreDeDecisaoNeural import ArvoreDeDecisaoNeural
from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2025-11-27 13:52:42.421824: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-27 13:52:42.422538: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-27 13:52:42.569518: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-27 13:52:45.632456: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

## Teste com uma base simples: problema do XOR

In [2]:
# Dados de exemplo: Porta XOR 
X = np.array([[0,0], [0,1], [1,0], [1,1]], dtype=float)
y = np.array([[0], [1], [1], [0]], dtype=float)

modelo_xor = tf.keras.Sequential([
    tf.keras.layers.Dense(7, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

modelo_xor.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.05), loss='mse')

modelo_xor.fit(X, y, epochs=100) # Treino rápido

/home/okiyud/UFAM/Disciplinas/Grafos/NN-as-DT/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-27 13:52:47.344808: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.6682
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.3951
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.2837
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.2402
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.2312
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.2364
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.2250
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.2015
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.1761
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.1614
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.1464
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.1322
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.1198
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.1073
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0973
Epoch 16/100
1/1 ━━━━

In [3]:
# Testar uma previsão real só para comparar
entrada_teste = np.array([[0,0], [0,1], [1,0], [1,1]])
previsao = modelo_xor.predict(entrada_teste, verbose=0)
for i, entrada in enumerate(entrada_teste):
    print(f"Previsão da Rede para {entrada}: {previsao[i][0]:.6f}")

Previsão da Rede para [0 0]: 0.002493
Previsão da Rede para [0 1]: 0.999821
Previsão da Rede para [1 0]: 1.000143
Previsão da Rede para [1 1]: 0.002493


In [4]:
arvore = ArvoreDeDecisaoNeural(modelo_xor)
for entrada in entrada_teste:
    previsao = arvore.prever(entrada)
    print(f"Previsão da Árvore para {entrada}: {previsao[0]:.6f}")

Previsão da Árvore para [0 0]: 0.002493
Previsão da Árvore para [0 1]: 0.999821
Previsão da Árvore para [1 0]: 1.000143
Previsão da Árvore para [1 1]: 0.002493


Mesmos resultados nas saídas dos modelos.

## Teste com dataset mais robusto: Breast Cancer

In [5]:
data = load_breast_cancer()
X = data.data      
y = data.target    

print(f"Dimensionalidade do dataset completo: X.shape = {X.shape}")
print(f"Classes: {np.unique(y)}, correspondente a {data.target_names}")

Dimensionalidade do dataset completo: X.shape = (569, 30)
Classes: [0 1], correspondente a ['malignant' 'benign']


In [6]:
# normalização da base, comum em tarefas de machine learning
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divide treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [7]:
modelo_breast_cancer = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(30,)), 
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear') 
])

modelo_breast_cancer.compile(optimizer='adam', 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=['accuracy'])

modelo_breast_cancer.fit(X_train, y_train, epochs=100)

/home/okiyud/UFAM/Disciplinas/Grafos/NN-as-DT/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.3363 - loss: 0.8502
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3912 - loss: 0.7448
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4220 - loss: 0.6637
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4879 - loss: 0.5895
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5648 - loss: 0.5184
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6967 - loss: 0.4485
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8000 - loss: 0.3880
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8725 - loss: 0.3329
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8989 - loss: 0.2886 
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9187 - loss: 0.2526
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9319 - loss: 0.2241 
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accura

In [8]:
loss, acuracia = modelo_breast_cancer.evaluate(X_test, y_test)
print(f"Acurácia do modelo de rede neural: {acuracia:.6f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9825 - loss: 0.0615 
Acurácia do modelo de rede neural: 0.982456


In [9]:
arvore = ArvoreDeDecisaoNeural(modelo_breast_cancer)

In [10]:
acertos = 0

for i in range(X_test.shape[0]):
    previsao = arvore.prever(X_test[i])

    # previsão positiva -> classe 1 (tem cancer)
    if previsao > 0 and y_test[i] == 1:
        acertos += 1
    # previsão não positiva -> classe 0 (não tem cancer)
    elif previsao <= 0 and y_test[i] == 0:
        acertos += 1 

acuracia = acertos / len(y_test)
print(f"Acurácia do modelo de árvore neural: {acuracia:.6f}")

Acurácia do modelo de árvore neural: 0.982456


Os modelos de árvore neural e rede neural possuem a mesma acurácia.

## Teste com problema multiclasse: Dataset Iris

In [11]:
data = load_iris()
X = data.data         
y = data.target    

print(f"Dimensionalidade do dataset completo: X.shape = {X.shape}")
print(f"Classes: {np.unique(y)}, correspondente a {data.target_names}")

Dimensionalidade do dataset completo: X.shape = (150, 4)
Classes: [0 1 2], correspondente a ['setosa' 'versicolor' 'virginica']


In [12]:
# normalização da base, comum em tarefas de machine learning
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divide treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [13]:
modelo_iris = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)), 
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax') 
])

modelo_iris.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
              metrics=['accuracy'])

modelo_iris.fit(X_train, y_train, epochs=50)

Epoch 1/50


/home/okiyud/UFAM/Disciplinas/Grafos/NN-as-DT/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.4917 - loss: 0.9408
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5167 - loss: 0.9189
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5250 - loss: 0.8997
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5917 - loss: 0.8811
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6333 - loss: 0.8639
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6667 - loss: 0.8476
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6833 - loss: 0.8319
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6833 - loss: 0.8174
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7000 - loss: 0.8035
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7250 - loss: 0.7906
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7417 - loss: 0.7780
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7417 - loss: 0.7659
Epoch 13/50


In [14]:
loss, acuracia = modelo_iris.evaluate(X_test, y_test)
print(f"Acurácia do modelo de rede neural: {acuracia:.6f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 0.8333 - loss: 0.4628
Acurácia do modelo de rede neural: 0.833333


In [15]:
arvore = ArvoreDeDecisaoNeural(modelo_iris)

In [16]:
acertos = 0

for i in range(X_test.shape[0]):
    previsao = arvore.prever(X_test[i])
    classe_predita = np.argmax(previsao)
    
    if classe_predita == y_test[i]:
        acertos += 1

acuracia = acertos / len(y_test)
print(f"Acurácia do modelo de árvore neural: {acuracia:.6f}")

Acurácia do modelo de árvore neural: 0.833333


Novamente, o modelo da árvore neural e da rede neural possuem a mesma acurácia!